In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

c:\Users\Broth\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('train.csv')

In [3]:
X = df['sms']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

КЛАССИЧЕСКАЯ МОДЕЛЬ (мультиномиальная наивная байесовская модель)

In [4]:
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
nb_model = MultinomialNB()
nb_model.fit(X_train_counts, y_train)
X_test_counts = vectorizer.transform(X_test)
nb_predictions = nb_model.predict(X_test_counts)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Accuracy of Naive Bayes model:", nb_accuracy)

Accuracy of Naive Bayes model: 0.9856502242152466


НС


In [5]:
model = Sequential([
    Dense(32, input_dim=X_train_counts.shape[1], activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_counts.toarray(), y_train, epochs=10, batch_size=32, validation_data=(X_test_counts.toarray(), y_test))

c:\Users\Broth\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8636 - loss: 0.4869 - val_accuracy: 0.9821 - val_loss: 0.1443
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9872 - loss: 0.1092 - val_accuracy: 0.9857 - val_loss: 0.0767
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9968 - loss: 0.0407 - val_accuracy: 0.9874 - val_loss: 0.0601
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9981 - loss: 0.0229 - val_accuracy: 0.9874 - val_loss: 0.0570
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9995 - loss: 0.0113 - val_accuracy: 0.9874 - val_loss: 0.0554
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9996 - loss: 0.0077 - val_accuracy: 0.9874 - val_loss: 0.0572
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9998 - loss: 0.0056 - val_accuracy: 0.9874 - val_loss: 0.0588
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9996 - loss: 0.0039 - val_accuracy: 0.

Тут я менял и версии, думал что несовместимость, и модель менял на DistilBert. Короче я не смог пофиксить проблему...

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = TFBertModel.from_pretrained('bert-base-uncased')

TypeError: 'NoneType' object is not callable

In [ ]:
X_train_bert = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors="tf")
X_test_bert = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors="tf")

ТРАНСФОРМЕР (на ноутбуке всё запускалось отлично, как видно из шага ниже этого (где происходит обучение, а именно model.fit), а на компе, где я и хотел обучать модель из-за большего кол-ва мощностей и возможности оставить его на ночь возникла проблема, которую я так и не смог решить)

In [ ]:
input_ids = tf.keras.layers.Input(shape=(X_train_bert['input_ids'].shape[1],), name='input_ids', dtype='int32')
outputs = TFBertModel.from_pretrained('bert-base-uncased')(input_ids)
pooled_output = outputs.pooler_output
output = tf.keras.layers.Dense(1, activation='sigmoid')(pooled_output)
model = tf.keras.Model(inputs=input_ids, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Вот тут видно, что на ноуте всё работало...

In [ ]:
model.fit(X_train_bert['input_ids'], y_train, epochs=3, batch_size=32, validation_data=(X_test_bert['input_ids'], y_test))

Epoch 1/3
  4/140 [..............................] - ETA: 3:50:47 - loss: 0.8132 - accuracy: 0.5234